# Laboratorio N°5 - Computación Científica II - ILI286

**Autores**:
+ Gonzalo Moya - 201173016-k 
+ Ariel Sanhueza     - 201173005-4

## Introducción

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Morbi venenatis dui felis. Donec ante libero, vestibulum nec faucibus sit amet, congue vel erat. Aliquam aliquet dolor mauris. Quisque egestas tincidunt aliquet. Integer tempor tellus quis commodo porttitor. Donec accumsan tellus sed leo sodales, ac malesuada lacus dignissim. Aliquam erat volutpat. Nunc laoreet vitae felis nec varius. Maecenas sed sem non quam faucibus tristique. Nullam scelerisque ante arcu, id faucibus orci tincidunt nec. Aliquam eu leo vehicula leo maximus vulputate. Donec eget nibh dolor. Aliquam eget ullamcorper erat. Nulla sodales, metus finibus venenatis tristique, elit neque pretium mi, eu maximus nibh turpis vulputate lectus. Phasellus vitae pharetra nisi. 

## Desarrollo

### Parte 2:  Diferencias finitas en 2D

### a)  Discretizacion forward

Para la discretizacion con un esquema forward, primero se presentan los elemenos que incluyen la derivación númerica para despues ser reemplazados en los sistemas correspondientes:
\begin{equation*}
{\Delta} u = \frac{u_{i-1,j,n}-2u_{i,j,n}+u_{i+1,j,n}}{{\Delta x}^2} + \frac{u_{i,j-1,n}-2u_{i,j,n}+u_{i,j+1,n}}{{\Delta y}^2}  \\
{\Delta} v = \frac{v_{i-1,j,n}-2v_{i,j,n}+v_{i+1,j,n}}{{\Delta x}^2} + \frac{v_{i,j-1,n}-2v_{i,j,n}+v_{i,j+1,n}}{{\Delta y}^2}  \\
\partial_t u = \frac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t} \\
\partial_t v = \frac{u_{i,j,n+1} - v_{i,j,n}}{\Delta t}.
\end{equation*}
Luego al ser utilizadas estas discretizaciones en el sistema que fue presentado y ademas discretizar el resto de los elemenos que dependan de $u$ y $v$, el restultado es:
\begin{align*}
\frac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t}  = 
D_{u} \frac{u_{i-1,j,n}-2u_{i,j,n}+u_{i+1,j,n}}{{\Delta x}^2} + D_u \frac{u_{i,j-1,n}-2u_{i,j,n}+u_{i,j+1,n}}{{\Delta y}^2} +
\lambda u_{i,j,n} - {u_{i,j,n}}^3 - \sigma v_{i,j,n} \\
\tau \frac{v_{i,j,n+1} - v_{i,j,n}}{\Delta t}  = 
D_{v} \frac{v_{i-1,j,n}-2v_{i,j,n}+v_{i+1,j,n}}{{\Delta x}^2} + D_v \frac{v_{i,j-1,n}-2v_{i,j,n}+v_{i,j+1,n}}{{\Delta y}^2} + u_{i,j,n}  - v_{i,j,n}.
\end{align*}
Finalmente se opera para re-agrupar términos semejantes y dejar ordenado el sistema:
\begin{align*}
u_{i,j,n+1} = \frac{\Delta t D_u}{\Delta x^2}u_{i+1,j,n} + \frac{\Delta t D_u}{\Delta x^2}u_{i-1,j,n} + 
\big( 1 - 2\frac{\Delta t D_u}{\Delta x^2} - 2\frac{\Delta t D_u}{\Delta y^2} + \Delta t \lambda \big)u_{i,j,n} + \Delta t {u_{i,j,k}}^3 \frac{\Delta t D_u}{\Delta y^2}u_{i,j+1,n} + \frac{\Delta t D_u}{\Delta y^2}u_{i,j-1,n} - \sigma v_{i,j,n} \\
\tau v_{i,j,n+1} = \frac{\Delta t D_v}{\Delta x^2}v_{i+1,j,n} + \frac{\Delta t D_v}{\Delta x^2}v_{i-1,j,n} + 
\big( \tau - 2\frac{\Delta t D_u}{\Delta x^2} - 2\frac{\Delta t D_u}{\Delta y^2} - \Delta t \big)v_{i,j,n} +  \frac{\Delta t D_v}{\Delta y^2}v_{i,j+1,n} + \frac{\Delta t D_v}{\Delta y^2}v_{i,j-1,n} + u_{i,j,n} \\ 
\end{align*}


### b)

In [22]:
def solve_forward(u0, v0, N, Tmax, Du, Dv, l, tau, sigma, k):
    Nx = N
    Ny = N
    dx = dy = 1./(N-1)
    dt = dx**2 - 0.00001
    Nt = Tmax/dt
    
    print dx, dt
    
    # Puntos
    x = np.linspace(0, 1, Nx)
    y = np.linspace(0, 1, Ny)
    t = np.linspace(0, Tmax, Nt)
    print len(x),len(y),len(t)
    
    # Soluciones
    sol_u = np.zeros((Nt, Nx, Ny))
    sol_v = np.zeros((Nt, Nx, Ny))
    
    # Valores constantes
    a_u = dt*Du/(dx**2)
    b_u = dt*Du/(dy**2)
    a_v = dt*Dv/(dx**2)
    b_v = dt*Dv/(dy**2)
    
    # condiciones iniciales u/v
    sol_u[0,:,:] = u0
    sol_v[0,:,:] = v0
    
    for n in xrange(1, len(t)):
        # Resto de casos
        Cu = np.zeros((sol_u.shape[1]+2, sol_u.shape[2]+2))
        Cu[1:-1,1:-1] = sol_u[n-1,:,:]
        Cu[-1,1:-1] = sol_u[n-1,1,:]
        Cu[0,1:-1] = sol_u[n-1,-2,:]
        Cu[1:-1,0] = sol_u[n-1,:,-2]
        Cu[1:-1,-1] = sol_u[n-1,:,1]

        Cv = np.zeros((sol_u.shape[1]+2, sol_u.shape[2]+2))
        Cv[1:-1,1:-1] = sol_v[n-1,:,:]
        Cv[-1,1:-1] = sol_v[n-1,1,:]
        Cv[0,1:-1] = sol_v[n-1,-2,:]
        Cv[1:-1,0] = sol_v[n-1,:,-2]
        Cv[1:-1,-1] = sol_v[n-1,:,1]

        for i in xrange(0, len(x)):
            for j in xrange(0, len(y)):
                sol_u[n,i,j] = a_u*Cu[i+1,j]+a_u*Cu[i-1,j]+(1-2*a_u-2*b_u+dt*l)*Cu[i,j]+b_u*Cu[i,j+1]+b_u*Cu[i,j-1]-dt*sigma*Cv[i,j]-dt*k-dt*(Cu[i,j]**3)
                sol_v[n,i,j] = a_v*Cv[i+1,j]+a_v*Cv[i-1,j]+(tau-2*a_v-2*b_v-dt)*Cv[i,j]+b_v*Cv[i,j+1]+b_v*Cv[i,j-1]+dt*Cu[i,j]
                sol_v[n,i,j] = sol_v[n,i,j]/tau
                if math.isnan(sol_u[n,i,j]):
                    print n,i,j
                    return sol_u
    
    return sol_u

### c)

### d)

# Parte 4: Simulación

In [23]:
N = 51
u0 = np.random.rand(N,N)
u0[:,0] = u0[:,-1] = np.random.rand(N)
u0[0,:] = u0[-1,:] = np.random.rand(N)

v0 = np.random.rand(N,N)
v0[:,0] = v0[:,-1] = np.random.rand(N)
v0[0,:] = v0[-1,:] = np.random.rand(N)

Du = 3e-4
Dv = 4e-3
l = 1.0
tau = 0.2
sigma = 1.0
k = -0.004

sol = solve_forward(u0, v0, N, 10, Du, Dv, l, tau, sigma, k)
print sol

0.02 0.00039
51 51 25641
[[[  4.09896505e-01   3.04157353e-01   9.37435924e-02 ...,   7.18895599e-01
     3.47762559e-01   5.05067218e-01]
  [  8.71608956e-01   7.21559806e-01   3.70302187e-01 ...,   2.20252535e-01
     1.69807230e-01   8.71608956e-01]
  [  6.53086890e-01   7.56133469e-01   1.57068457e-01 ...,   7.06206311e-01
     7.78680715e-01   6.53086890e-01]
  ..., 
  [  1.38593662e-01   8.45807043e-01   4.26787555e-01 ...,   9.38279952e-01
     5.04830967e-01   1.38593662e-01]
  [  4.33523877e-01   6.75462330e-01   8.01074554e-01 ...,   1.13570653e-01
     4.12435984e-02   4.33523877e-01]
  [  4.09896505e-01   3.04157353e-01   9.37435924e-02 ...,   7.18895599e-01
     3.47762559e-01   5.05067218e-01]]

 [[  2.30086283e-04   4.33715464e-01   6.75402690e-01 ...,   8.12631744e-02
     1.13439422e-01   4.12037070e-02]
  [  3.47707910e-01   4.09923141e-01   3.04277981e-01 ...,   7.50757951e-01
     7.18296683e-01   3.47869122e-01]
  [  1.70263032e-01   8.70857161e-01   7.21482384e-01

In [27]:
animate_pattern(sol)

## Conclusiones

+ Nullam tincidunt commodo arcu et consectetur. Duis iaculis, risus vel scelerisque interdum, ante nunc varius urna, nec accumsan ex purus vitae turpis. Etiam ut egestas ligula, id lacinia nunc. 
+ Praesent sodales ligula eu lorem placerat convallis. Maecenas elementum venenatis maximus. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Proin vitae finibus nulla, vel auctor orci. Ut et sodales lectus, nec sodales nunc. 
+ Quisque laoreet feugiat diam, a lacinia felis pharetra eu. Donec feugiat elit sapien, a cursus diam hendrerit in.
+ Etiam porta ipsum a aliquet hendrerit. Pellentesque fringilla dolor pulvinar lorem ullamcorper, non vestibulum nulla volutpat. Sed ac diam fringilla tortor iaculis tincidunt. 
+ Integer sapien augue, accumsan nec sagittis at, sagittis quis quam. Maecenas consectetur metus non tellus laoreet, nec efficitur lacus venenatis. Nullam ac dolor quis orci pretium commodo. 
+ Maecenas eget ex in lectus tincidunt lobortis. Donec non gravida nisl. Fusce sagittis sagittis neque consectetur cursus. Duis commodo mauris ligula, eu aliquam purus porta quis. 
+ Suspendisse euismod mi lorem, in ultricies neque interdum sit amet. Vivamus ut ullamcorper tortor, vel faucibus risus. Curabitur malesuada lorem diam. Nam ornare nisi id sapien viverra, semper commodo dolor eleifend. 

## Anexos

En la siguiente sección se encuentra todo el código necesario para reproducir cada uno de los resultados mostrados anteriormente. Para poder ejecutar el código en el informe, se debe en primer lugar ejecutar las celdas de código presentes en este anexo.

### Configuración del notebook

In [68]:
#notebook settings
#%matplotlib inline
#%matplotlib qt

#import some useful libraries and utilities
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math

In [69]:
"""
> sol must be a 3D numpy array, with dimensions (Nt,Nx,Ny)
> time_step:  miliseconds to update canvas
> frame_step: Jump step for frames to show
"""
def animate_pattern(sol, time_step=10, frame_step=20):
    #time index
    t_ind = 0
    fig = plt.figure()
    im = plt.imshow(sol[t_ind], cmap=plt.cm.winter)
    plt.xticks([]); plt.yticks([])

    #setting the number of frames
    frames = int(sol.shape[0]/frame_step)
    
    #update function
    def updatefig(t_ind):
        im = plt.imshow(sol[t_ind*frame_step], cmap=plt.cm.winter)
        return im,
    
    #animate it!
    ani = animation.FuncAnimation(fig, updatefig, frames=frames, 
                    interval=time_step, blit=True, repeat=True)
    plt.show()

In [73]:
X = 50
#A = np.random.rand(X**3,X,X)
print A.shape
B = np.random.random((27777,50,50))

(125000, 50, 50)


In [74]:
animate_pattern(A)